In [ ]:
#import necessary libraries

import numpy as np
import pandas as pd 
import nltk

from gensim.models import Word2Vec
from nltk.corpus import stopwords

import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize

Here,I am using Tensorfolw and karas instead of Pytorch, to learn and experiment these libraries as well 

In [ ]:
# Load data

train_data = pd.read_csv('../input/devdata/cw2_train.csv')
dev_data = pd.read_csv('../input/devdata/cw2_dev.csv')#test data

train_data.head()

,Unnamed: 0,Sent1,Sent2,SimScore
0,0,"U.S., EU Widen Sanctions On Russia","U.S., EU Boost Sanctions On Russia",1.00
1,1,The lawyers advised the judges .,The lawyers advised the judges behind the acto...,0.79
2,2,Man kills 4 in Calif. before police shoot him ...,Police: Gunman killed 6 in California shootings,0.40
3,3,Someone is playing a piano.,A man is playing a guitar.,0.24
4,4,In an E-mail statement to the Knoxville News S...,I am not giving any consideration to resignati...,0.80


## Text Cleaning

In [ ]:
corpus1 = []
for i in train_data.Sent1:
    
    review = re.sub('[^a-zA-Z]', ' ', i )
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus1.append(review)

In [ ]:
corpus2 = []
for i in train_data.Sent2:
    
    review = re.sub('[^a-zA-Z]', ' ', i )
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus2.append(review)

Marge sent1 and sent2

In [ ]:
from pandas import DataFrame
df = DataFrame (corpus1,columns=['c1'])
df['c2'] = corpus2

In [ ]:
a=list(df.c1.values.astype(str))+list(df.c2.values.astype(str))

In [ ]:
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(list(df.c1.values.astype(str))+list(df.c2.values.astype(str)))

#### Padding and sequencing

In [ ]:
X_train1 = tokenizer.texts_to_sequences(df.c1.values.astype(str))
X_train1 = pad_sequences(X_train1, maxlen = 10, padding='post')


X_train2 = tokenizer.texts_to_sequences(df.c2.values.astype(str))
X_train2 = pad_sequences(X_train2, maxlen = 10, padding='post')

In [ ]:
word_index = tokenizer.word_index

word_index
'the': 1,
 'a': 2,
 'in': 3,
 'is': 4,
 'to': 5,
 'and': 6,
 'of': 7,
 'on': 8,
 'that': 9,
 'man': 10,
 'president': 11,
 'by': 12,

## Word embading

In [ ]:
embedding_index = {}
with open('../input/glove6b300dtxt/glove.6B.300d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [ ]:
embedding_matrix = np.random.random((len(word_index)+1, 300))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## LSTM

add Codeadd Markdown
Long Short Term Memory is a variation of RNN which is used to eliminate the Vanishing Gradients Problem. It is much more powerful and complex than other variations of RNN, i.e., GRU.

For RNN(LSTM) tanh is default activation funtion still I am writting for understanding purpose 


In [ ]:
# Model for S1
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization


model_q1 = tf.keras.Sequential()
model_q1.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 300,
                      weights = [embedding_matrix],
                      input_length = 10))
model_q1.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q1.add(Dropout(0.1))
model_q1.add(LSTM(128, return_sequences = True))
model_q1.add(LSTM(128))
model_q1.add(Dense(60, activation = 'tanh'))
model_q1.add(Dense(1, activation = 'sigmoid'))

In [ ]:
# Model for S2
model_q2 = tf.keras.Sequential()
model_q2.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 300,
                      weights = [embedding_matrix],
                      input_length = 10))
model_q2.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q2.add(Dropout(0.1))
model_q2.add(LSTM(128, return_sequences = True))
model_q2.add(LSTM(128))
model_q2.add(Dense(60, activation = 'tanh'))
model_q2.add(Dense(1, activation = 'sigmoid'))

In [ ]:
# Merging the output of the two models,i.e, model_q1 and model_q2
mergedOut = Multiply()([model_q1.output, model_q2.output])
    
mergedOut = Flatten()(mergedOut)
mergedOut = Dense(100, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.1)(mergedOut)
mergedOut = Dense(50, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.1)(mergedOut)
mergedOut = Dense(1, activation = 'sigmoid')(mergedOut)

I tried with different architecture and I found this architecture gives best performance,
after that next is fit the model,for that I used different optimizer like GD,SGD then finally adam give better perfomance,then I tried different batch_size like 2000,3000,4000 and last but not least, I found epoch 40 , it can be give better performace if epoch is more than 40,but it would be time cosuming and require more computation power as well.

In [ ]:
new_model = Model([model_q1.input, model_q2.input], mergedOut)
new_model.compile(optimizer = 'adam', loss = 'MeanSquaredError', metrics=[tf.keras.metrics.Accuracy()])
history = new_model.fit([X_train1,X_train2],train_data['SimScore'], batch_size = 2000, epochs = 40)

Epoch 1/40
6/6 [==============================] - 20s 2s/step - loss: 0.0675
Epoch 2/40
6/6 [==============================] - 9s 1s/step - loss: 0.0666
Epoch 3/40
6/6 [==============================] - 8s 1s/step - loss: 0.0663
Epoch 4/40
6/6 [==============================] - 8s 1s/step - loss: 0.0654
Epoch 5/40
6/6 [==============================] - 8s 1s/step - loss: 0.0650
Epoch 6/40
6/6 [==============================] - 8s 1s/step - loss: 0.0639
Epoch 7/40
6/6 [==============================] - 8s 1s/step - loss: 0.0638
Epoch 8/40
6/6 [==============================] - 8s 1s/step - loss: 0.0642
Epoch 9/40
6/6 [==============================] - 8s 1s/step - loss: 0.0639
Epoch 10/40
6/6 [==============================] - 8s 1s/step - loss: 0.0620
Epoch 11/40
6/6 [==============================] - 8s 1s/step - loss: 0.0622
Epoch 12/40
6/6 [==============================] - 8s 1s/step - loss: 0.0610
Epoch 13/40
6/6 [==============================] - 8s 1s/step - loss: 0.0607
Epoch 1

In [ ]:
#instead of pickel,here I am using karas "save" function
new_model.save("my_model")


In [ ]:
#Load the model
d_model = keras.models.load_model("./my_model")

In [ ]:
!zip -r file.zip ./my_model